In [1]:
import pandas as pd
import matplotlib.pyplot as plt

import pyodbc
import numpy as np

In [2]:
df = pd.read_csv('input/2023Jan1-2023Nov30AmznSalesReportR.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45981 entries, 0 to 45980
Data columns (total 30 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   date/time                 45981 non-null  object 
 1   settlement id             45981 non-null  int64  
 2   type                      45981 non-null  object 
 3   order id                  45609 non-null  object 
 4   sku                       33477 non-null  object 
 5   description               45981 non-null  object 
 6   quantity                  33325 non-null  float64
 7   marketplace               33422 non-null  object 
 8   account type              45981 non-null  object 
 9   fulfillment               33100 non-null  object 
 10  order city                33094 non-null  object 
 11  order state               33054 non-null  object 
 12  order postal              33093 non-null  object 
 13  tax collection model      30232 non-null  object 
 14  produc

**Convert columns to numeric**

In [3]:
df['product sales']=df['product sales'].apply(lambda x: x.replace(',',''))
df['product sales'] = df['product sales'].astype(float)

In [4]:
df['other']=df['other'].apply(lambda x: x.replace(',',''))
df['other'] = df['other'].astype(float)

In [5]:
df['total']=df['total'].apply(lambda x: x.replace(',',''))
df['total'] = df['total'].astype(float)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45981 entries, 0 to 45980
Data columns (total 30 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   date/time                 45981 non-null  object 
 1   settlement id             45981 non-null  int64  
 2   type                      45981 non-null  object 
 3   order id                  45609 non-null  object 
 4   sku                       33477 non-null  object 
 5   description               45981 non-null  object 
 6   quantity                  33325 non-null  float64
 7   marketplace               33422 non-null  object 
 8   account type              45981 non-null  object 
 9   fulfillment               33100 non-null  object 
 10  order city                33094 non-null  object 
 11  order state               33054 non-null  object 
 12  order postal              33093 non-null  object 
 13  tax collection model      30232 non-null  object 
 14  produc

**Only orders with an Order Id (Drops various seperate fees(Service Fees, Adjustments, FBA Inv Fees))**

In [7]:
Oid = df.loc[~df['order id'].isnull()]
Oid.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45609 entries, 0 to 45980
Data columns (total 30 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   date/time                 45609 non-null  object 
 1   settlement id             45609 non-null  int64  
 2   type                      45609 non-null  object 
 3   order id                  45609 non-null  object 
 4   sku                       33334 non-null  object 
 5   description               45609 non-null  object 
 6   quantity                  33182 non-null  float64
 7   marketplace               33321 non-null  object 
 8   account type              45609 non-null  object 
 9   fulfillment               33100 non-null  object 
 10  order city                33094 non-null  object 
 11  order state               33054 non-null  object 
 12  order postal              33093 non-null  object 
 13  tax collection model      30232 non-null  object 
 14  product sal

**Select fulfillment**

All FBM 'Shipping Services' items get dropped as they dont have a fulfillment

In [8]:
fulfillment = 'Amazon'
fulfil = Oid.loc[Oid['fulfillment']==fulfillment]
fulfil

,date/time,settlement id,type,order id,sku,description,quantity,marketplace,account type,fulfillment,...,Regulatory Fee,Tax On Regulatory Fee,promotional rebates,promotional rebates tax,marketplace withheld tax,selling fees,fba fees,other transaction fees,other,total
16,"Jan 1, 2023 8:13:26 PM PST",17143526261,Order,113-0597596-1010624,794296966fba,Markal - 96966 Pro-Line HP High Performance Li...,1.0,amazon.com,Invoiced Orders,Amazon,...,0,0,0.00,0,-1.68,-4.20,-5.06,0.0,0.0,18.73
17,"Jan 1, 2023 11:09:19 PM PST",17149362761,Order,114-1237401-5494660,073134801fba,PIP 34-800/XL MaxiFoam G-Tek Premium Nitrile F...,1.0,amazon.com,Standard Orders,Amazon,...,0,0,0.00,0,-3.43,-7.35,-5.71,0.0,0.0,35.93
19,"Jan 2, 2023 1:35:02 AM PST",17149362761,Order,112-8442762-2924213,806088471fba,"3M Xtract Cubitron II Net Disc 710W, 120+, 5 i...",1.0,amazon.com,Standard Orders,Amazon,...,0,0,0.00,0,0.00,-3.30,-5.06,0.0,0.0,13.63
21,"Jan 2, 2023 4:29:46 AM PST",17149362761,Order,114-6930593-8312219,806088455fba,"3M Xtract Cubitron II Net Disc 710W, 80+, 6 in...",1.0,amazon.com,Standard Orders,Amazon,...,0,0,0.00,0,-1.51,-4.80,-5.71,0.0,0.0,21.48
22,"Jan 2, 2023 4:53:50 AM PST",17149362761,Order,112-8442762-2924213,806088476fba,"3M Xtract Cubitron II Net Disc 710W, 320+, 5 i...",1.0,amazon.com,Standard Orders,Amazon,...,0,0,0.00,0,0.00,-3.30,-4.27,0.0,0.0,14.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45976,"Nov 30, 2023 11:38:44 PM PST",19201956281,Order,111-1594656-8635414,806088489fba,"3M Xtract Cubitron II Net Disc 710W, 220+, 3 i...",1.0,amazon.com,Standard Orders,Amazon,...,0,0,0.00,0,-1.51,-3.12,-4.16,0.0,0.0,13.49
45977,"Nov 30, 2023 11:38:44 PM PST",19201956281,Order,113-2626300-0465837,147512162fba,"Lufkin Shockforce G2 16-ft Nite Eye, Magnetic ...",1.0,amazon.com,Standard Orders,Amazon,...,0,0,0.00,0,-1.75,-3.74,-5.70,0.0,0.0,15.50
45978,"Nov 30, 2023 11:41:29 PM PST",19201956281,Order,114-4344818-4509042,261185195fba,"GEARWRENCH 3/8"" 120XP Flex Head Electronic Tor...",1.0,amazon.com,Standard Orders,Amazon,...,0,0,-8.56,0,-13.19,-29.31,-11.99,0.0,0.0,154.12
45979,"Nov 30, 2023 11:54:36 PM PST",19201956281,Order,111-6046110-5737833,806088456fba,"Cubitron 3M Xtract Net Disc 710W, 120+, 6 in, ...",1.0,amazon.com,Standard Orders,Amazon,...,0,0,0.00,0,-3.21,-4.70,-5.70,0.0,0.0,20.96


In [9]:
if fulfillment == 'Amazon':
    fulfil['sku']=fulfil['sku'].apply(lambda x: str(x)[:9])
    fulfil['sku'].apply(lambda x: len(x)).unique()

C:\Users\rrichesin\AppData\Local\Temp\ipykernel_6056\2346058503.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fulfil['sku']=fulfil['sku'].apply(lambda x: str(x)[:9])


**Read in Cost sheet**

In [10]:
ordCost = pd.read_excel('input/amznCostOrders.xlsx')

In [11]:
ordCost.iloc[:,7:25].sample(10)

,Order Suffix,Warehouse,Customer PO,Reference,Line Number,Entered Date,Shipped Date,Invoice Date,Transaction Type,Customer Product Number,Product Number,"Stock, Special, Non-Stock",Description,Description 2,Qty Shipped,UOM,Unit Cost,Unit Sell Price
527,0,201,112-4116146-3084228,NaN,1,2023-01-05,2023-01-09,2023-01-09,SO,NaN,806088475,NaN,"5""XNH 710W 240+ XTRACT",CUBITRON NET DISC,3,CT,12.90560,22.99
5835,0,201,113-7983802-6693827,NaN,1,2023-03-05,2023-03-07,2023-03-07,SO,NaN,794296960,NaN,96960 PRO-LINE HP WHITE,PAINT MARKER,1,BX,13.92000,32.99
7637,0,201,114-8492820-2488223,NaN,1,2023-03-23,2023-03-24,2023-03-24,SO,NaN,261185195,NaN,85195 TORQUE WRENCH FLEX,HEAD 3/8 10-100 FT LB,1,EA,148.29090,224.99
18332,0,901A,114-1152845-2038669,NaN,1,2023-07-31,2023-07-31,2023-07-31,SO,NaN,806088471,s,"5""XNH 710W 120+ XTRACT",CUBITRON NET DISC,1,CT,11.69226,21.60
3802,0,201,111-6050127-1638651,NaN,1,2023-02-16,2023-02-17,2023-02-17,SO,NaN,794296823,NaN,96823 BLACK VALVE ACTION,PAINT MARKER,1,BX,13.67100,25.99
19752,0,901A,112-7027153-1991411,NaN,1,2023-08-14,2023-08-14,2023-08-14,SO,NaN,794296270,s,96270 PRO HOLDER,STRT LEAD,1,EA,11.05000,23.00
8510,0,201,114-1084978-6797059,NaN,2,2023-04-04,2023-04-05,2023-04-05,SO,NaN,806088456,NaN,"6""XNH 710W 120+ XTRACT",CUBITRON NET DISC,1,CT,21.46870,31.99
26401,0,201,112-8068502-0797813,NaN,1,2023-11-01,2023-11-02,2023-11-02,SO,NaN,806088470,NaN,"5""XNH 710W 80+ XTRACT",CUBITRON DISC 66580,1,CT,15.18300,21.14
26633,0,201,113-8552740-0457061,NaN,1,2023-11-05,2023-11-06,2023-11-06,SO,NaN,806088470,NaN,"5""XNH 710W 80+ XTRACT",CUBITRON DISC 66580,1,CT,15.18300,26.13
10559,0,901A,111-4470021-0862629,NaN,1,2023-04-26,2023-04-26,2023-04-26,SO,NaN,134014751,s,DGR51 .5 HP RIGHT ANGLE,DIE GRINDER,5,EA,74.26000,127.50


In [12]:
ordCost.rename(columns={'Product Number':'sku'}, inplace=True)
cost = ordCost[['sku', 'Unit Cost']]
cost.sample(5)

,sku,Unit Cost
3580,261185196,181.14956
3100,261185034,60.00000
10959,806088474,13.66894
31291,806088458,18.66174
7698,794296820,13.64796


**Drop returned orders from cost sheet**

In [13]:
cost = cost.loc[cost['Unit Cost']>0]
cost['sku'].nunique()

239

**Groupby cost to get average cost per item (since cost changes over time and whse)**

In [14]:
cost_gb = pd.DataFrame(cost.groupby('sku')['Unit Cost'].mean().reset_index(),columns=['sku','Unit Cost'])

In [15]:
cost_gb['sku'] = cost_gb['sku'].astype(str)
cost_gb['sku'] = cost_gb['sku'].str.zfill(9)

In [16]:
cost_gb

,sku,Unit Cost
0,001205841,2.840000
1,014297675,314.145000
2,030424352,144.550000
3,032003400,36.163845
4,056200331,12.830440
...,...,...
234,986800331,27.443270
235,986800341,38.575200
236,986803410,22.723793
237,434-61053,70.748000


**Combine to add product costs to df**

In [17]:
both = fulfil.join(cost_gb.set_index('sku'), on='sku')

In [18]:
both.sample(5)

,date/time,settlement id,type,order id,sku,description,quantity,marketplace,account type,fulfillment,...,Tax On Regulatory Fee,promotional rebates,promotional rebates tax,marketplace withheld tax,selling fees,fba fees,other transaction fees,other,total,Unit Cost
35733,"Sep 20, 2023 4:03:59 PM PDT",18749537461,Order,113-5074488-2648269,806088456,"Cubitron 3M Xtract Net Disc 710W, 120+, 6 in, ...",1.0,amazon.com,Standard Orders,Amazon,...,0,0.0,0,0.00,0.00,-5.40,0.0,0.0,30.66,20.315975
22512,"Jun 4, 2023 3:18:00 AM PDT",18054338581,Order,111-1028744-0432247,806088456,"3M Xtract Cubitron II Net Disc 710W, 120+, 6 i...",1.0,amazon.com,Standard Orders,Amazon,...,0,0.0,0,-2.64,-4.80,-5.40,0.0,0.0,21.79,20.315975
9446,"Mar 2, 2023 10:25:37 AM PST",17466329531,Order,112-8575530-7315414,806088462,"3M Xtract Cubitron II Net Disc 710W, 320+, 6 i...",1.0,amazon.com,Standard Orders,Amazon,...,0,0.0,0,-1.92,-4.80,-5.40,0.0,0.0,21.79,19.920409
27643,"Jul 18, 2023 9:16:15 PM PDT",18308170881,Order,112-6208539-0172246,806088470,"3M Xtract Cubitron II Net Disc 710W, 80+, 5 in...",1.0,amazon.com,Standard Orders,Amazon,...,0,0.0,0,-1.82,-3.29,-4.75,0.0,0.0,13.86,12.170029
42023,"Nov 1, 2023 11:58:00 AM PDT",19015309651,Order,113-8669414-1012230,806088471,"Cubitron 3M Xtract Net Disc 710W, 120+, 5 in, ...",1.0,amazon.com,Standard Orders,Amazon,...,0,0.0,0,-1.99,-3.36,-5.05,0.0,0.0,13.96,11.728623


**Get total cost of goods per order**

In [19]:
both['Extended Cost'] = both['quantity'] * both['Unit Cost']
both.iloc[:,14:]

,product sales,product sales tax,shipping credits,shipping credits tax,gift wrap credits,giftwrap credits tax,Regulatory Fee,Tax On Regulatory Fee,promotional rebates,promotional rebates tax,marketplace withheld tax,selling fees,fba fees,other transaction fees,other,total,Unit Cost,Extended Cost
16,27.99,1.68,0.00,0.0,0.0,0.0,0,0,0.00,0,-1.68,-4.20,-5.06,0.0,0.0,18.73,15.366171,15.366171
17,48.99,3.43,0.00,0.0,0.0,0.0,0,0,0.00,0,-3.43,-7.35,-5.71,0.0,0.0,35.93,30.322755,30.322755
19,21.99,0.00,0.00,0.0,0.0,0.0,0,0,0.00,0,0.00,-3.30,-5.06,0.0,0.0,13.63,11.728623,11.728623
21,31.99,1.51,0.00,0.0,0.0,0.0,0,0,0.00,0,-1.51,-4.80,-5.71,0.0,0.0,21.48,19.302393,19.302393
22,21.99,0.00,0.00,0.0,0.0,0.0,0,0,0.00,0,0.00,-3.30,-4.27,0.0,0.0,14.42,11.750324,11.750324
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45976,20.77,1.51,0.00,0.0,0.0,0.0,0,0,0.00,0,-1.51,-3.12,-4.16,0.0,0.0,13.49,8.732157,8.732157
45977,24.94,1.75,0.00,0.0,0.0,0.0,0,0,0.00,0,-1.75,-3.74,-5.70,0.0,0.0,15.50,11.840000,11.840000
45978,195.42,13.19,8.56,0.0,0.0,0.0,0,0,-8.56,0,-13.19,-29.31,-11.99,0.0,0.0,154.12,139.657733,139.657733
45979,31.36,3.21,0.00,0.0,0.0,0.0,0,0,0.00,0,-3.21,-4.70,-5.70,0.0,0.0,20.96,20.315975,20.315975


**Get refund totals for fulfillment type**

In [20]:
refunds = df.loc[(df['fulfillment']==fulfillment) & (df['type']=='Refund')]

**Drop refunds from amzn file**

In [21]:
bothNoRefunds = both.loc[both['type']=='Order']

In [22]:
bothNoRefunds

,date/time,settlement id,type,order id,sku,description,quantity,marketplace,account type,fulfillment,...,promotional rebates,promotional rebates tax,marketplace withheld tax,selling fees,fba fees,other transaction fees,other,total,Unit Cost,Extended Cost
16,"Jan 1, 2023 8:13:26 PM PST",17143526261,Order,113-0597596-1010624,794296966,Markal - 96966 Pro-Line HP High Performance Li...,1.0,amazon.com,Invoiced Orders,Amazon,...,0.00,0,-1.68,-4.20,-5.06,0.0,0.0,18.73,15.366171,15.366171
17,"Jan 1, 2023 11:09:19 PM PST",17149362761,Order,114-1237401-5494660,073134801,PIP 34-800/XL MaxiFoam G-Tek Premium Nitrile F...,1.0,amazon.com,Standard Orders,Amazon,...,0.00,0,-3.43,-7.35,-5.71,0.0,0.0,35.93,30.322755,30.322755
19,"Jan 2, 2023 1:35:02 AM PST",17149362761,Order,112-8442762-2924213,806088471,"3M Xtract Cubitron II Net Disc 710W, 120+, 5 i...",1.0,amazon.com,Standard Orders,Amazon,...,0.00,0,0.00,-3.30,-5.06,0.0,0.0,13.63,11.728623,11.728623
21,"Jan 2, 2023 4:29:46 AM PST",17149362761,Order,114-6930593-8312219,806088455,"3M Xtract Cubitron II Net Disc 710W, 80+, 6 in...",1.0,amazon.com,Standard Orders,Amazon,...,0.00,0,-1.51,-4.80,-5.71,0.0,0.0,21.48,19.302393,19.302393
22,"Jan 2, 2023 4:53:50 AM PST",17149362761,Order,112-8442762-2924213,806088476,"3M Xtract Cubitron II Net Disc 710W, 320+, 5 i...",1.0,amazon.com,Standard Orders,Amazon,...,0.00,0,0.00,-3.30,-4.27,0.0,0.0,14.42,11.750324,11.750324
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45976,"Nov 30, 2023 11:38:44 PM PST",19201956281,Order,111-1594656-8635414,806088489,"3M Xtract Cubitron II Net Disc 710W, 220+, 3 i...",1.0,amazon.com,Standard Orders,Amazon,...,0.00,0,-1.51,-3.12,-4.16,0.0,0.0,13.49,8.732157,8.732157
45977,"Nov 30, 2023 11:38:44 PM PST",19201956281,Order,113-2626300-0465837,147512162,"Lufkin Shockforce G2 16-ft Nite Eye, Magnetic ...",1.0,amazon.com,Standard Orders,Amazon,...,0.00,0,-1.75,-3.74,-5.70,0.0,0.0,15.50,11.840000,11.840000
45978,"Nov 30, 2023 11:41:29 PM PST",19201956281,Order,114-4344818-4509042,261185195,"GEARWRENCH 3/8"" 120XP Flex Head Electronic Tor...",1.0,amazon.com,Standard Orders,Amazon,...,-8.56,0,-13.19,-29.31,-11.99,0.0,0.0,154.12,139.657733,139.657733
45979,"Nov 30, 2023 11:54:36 PM PST",19201956281,Order,111-6046110-5737833,806088456,"Cubitron 3M Xtract Net Disc 710W, 120+, 6 in, ...",1.0,amazon.com,Standard Orders,Amazon,...,0.00,0,-3.21,-4.70,-5.70,0.0,0.0,20.96,20.315975,20.315975


In [23]:
bothNoRefunds['order id'].nunique()

14791

# Sums

In [24]:
sums = bothNoRefunds.iloc[:,14:].sum()
sums = pd.DataFrame(sums, columns=['sum'])

**adding the additional fba(Adjustments, FBA Inv, FBA Refunds) or fbm (FBM Refunds, Shipping Services) specific fees**

In [25]:
if fulfillment == 'Amazon':
    sums.loc['Adjustments'] = df.loc[df['type']=='Adjustment'].iloc[:,14:].sum()['total']
    sums.loc['FBA Inventory Fees'] = df.loc[df['type']=='FBA Inventory Fee'].iloc[:,14:].sum()['total']
    sums.loc['Refunds'] = refunds.iloc[:,14:].sum()['total']
elif fulfillment == 'Seller':
    sums.loc['Shipping Services'] = df.loc[df['type']=='Shipping Services'].iloc[:,14:].sum()['total']
    sums.loc['Refunds'] = refunds.iloc[:,14:].sum()['total']

In [26]:
refunds.iloc[:,14:].sum()['total']

-15869.759999999998

**Net profit**

In [27]:
sums.loc['total']['sum'] - sums.loc['Extended Cost']['sum']

112318.1060856639

**Net profit less specific fees**

In [28]:
net = sums.loc['total']['sum'] - sums.loc['Extended Cost']['sum']
if fulfillment == 'Amazon':
    net2 = net + sums.loc['Adjustments']['sum'] + sums.loc['FBA Inventory Fees']['sum'] + sums.loc['Refunds']['sum']
    print(net + sums.loc['Adjustments']['sum'] + sums.loc['FBA Inventory Fees']['sum'] + sums.loc['Refunds']['sum'])
else:
    net2 = net + sums.loc['Shipping Services']['sum'] + sums.loc['Refunds']['sum']
    print(net + sums.loc['Shipping Services']['sum'] + sums.loc['Refunds']['sum'])

93197.63608566391


**Adding net profit row**

In [29]:
sums.loc['Net Profit'] = net2

In [30]:
#sums

**Adds percentage of sales values**

In [31]:
sums['percentage of sales'] = sums['sum'].apply(lambda x: round(((x / sums.loc['product sales']) * 100),1))
sums

,sum,percentage of sales
product sales,757939.160000,100.0
product sales tax,48865.480000,6.4
shipping credits,7639.290000,1.0
shipping credits tax,109.520000,0.0
gift wrap credits,11.980000,0.0
giftwrap credits tax,0.250000,0.0
Regulatory Fee,0.000000,0.0
Tax On Regulatory Fee,0.000000,0.0
promotional rebates,-5330.060000,-0.7
promotional rebates tax,0.000000,0.0


In [32]:
#sums.to_excel('fbm2.xlsx')

In [37]:
df.loc[(df['type']=='Shipping Services') & (df['description']=='Shipping Label Purchased through Amazon')]['total'].sum()

-114878.17000000001

In [38]:
df.loc[(df['type']=='Shipping Services') & (df['description']=='Adjustment')]['total'].sum()

-778.58

In [58]:
df.loc[df['type']=='Adjustment']['description'].unique()

array(['FBA Inventory Reimbursement - General Adjustment',
       'FBA Inventory Reimbursement - Customer Return',
       'FBA Inventory Reimbursement - Customer Service Issue',
       'FBA Inventory Reimbursement - Damaged:Warehouse',
       'Non-subscription Fee Adjustment'], dtype=object)

In [59]:
dic = {}
for i in df.loc[df['type']=='Adjustment']['description'].unique():
    val = round(df.loc[(df['type']=='Adjustment') & (df['description']==i)]['total'].sum(),2)
    sums.loc[i]= val
    dic[i]= val
    

In [60]:
dic

{'FBA Inventory Reimbursement - General Adjustment': -992.24,
 'FBA Inventory Reimbursement - Customer Return': 2618.72,
 'FBA Inventory Reimbursement - Customer Service Issue': 219.56,
 'FBA Inventory Reimbursement - Damaged:Warehouse': 3010.5,
 'Non-subscription Fee Adjustment': -200.49}

In [62]:
include = ['product sales', 'shipping credits', 'promotional rebates', 'selling fees',
           'fba fees', 'total' ,'cost of goods']
dic = {}
sums.loc['fba inventory fees'] = df.loc[df['type']=='FBA Inventory Fee'][include[:6]].sum()['total']
#add fba external fees
for i in df.loc[df['type']=='FBA Inventory Fee']['description'].unique():
    val = round(df.loc[(df['type']=='Adjustment') & (df['description']==i)]['total'].sum(),2)
    dic[i]= val

In [63]:
dic

{'FBA Amazon-Partnered Carrier Shipment Fee': 0.0,
 'FBA Inventory Storage Fee': 0.0,
 'FBA Removal Order: Return Fee': 0.0,
 'FBA Long-Term Storage Fee': 0.0}

In [64]:
df.loc[(df['type']=='Adjustment') & (df['description']==i)]

,date/time,settlement id,type,order id,sku,description,quantity,marketplace,account type,fulfillment,...,Regulatory Fee,Tax On Regulatory Fee,promotional rebates,promotional rebates tax,marketplace withheld tax,selling fees,fba fees,other transaction fees,other,total
